# Rabbit Image Downloader

#### To train my neural network I had 5000 pictures each of dogs and cats but none of rabbits. The most obvious answer to finding 5000 images of rabbits was to scrape them off of Google Images. After looking through a few solutions, I found that this one from Anand Suresh which uses Selenium worked the best. 

## Problem
#### The challenge I faced using this was an arbitrary download limit of around 125ish images per query. Some research led me down a rabbit hole of changes that Google pushed out to their search platforms (including image) relatively recently (mid 2020) while integrating BERT their new NLP model (link below). 

## Solution
#### I wanted to get my feet wet with the neural network so instead of trying to figure out a way to query 5000 images (or any number higher than 150) I figured I could get similar results by just querying 50 different species of rabbit. This had the added benefit of making sure that the images i scraped were of a diversity of types so that my neural network would be more robust. Eventually I ran out of species and started querying the word "rabbit" in multiple languages, the theory being that websites where rabbits were tagged in their native languages might have different images of rabbits anyways.

#### Of course in the process of scraping this many images I would come up with some images that were low quality or duplicates. I used the free Mac app, "dupeguru" to get rid of duplicate images and manually checked the images to make sure that they were of rabbits and  could be easily identified as rabbits. 

In [49]:
from tqdm import tqdm_notebook

search_terms = ["rabbit", "bunny", "hare", "coney", "cute rabbit", "adopt rabbit", "english lop", "jersey wooly", "checkered giant rabbit", "mini rex", "blan de hotot", "belgian hare", "flemish giant rabbit", "mini lop", "californian rabbit", "american rabbit", "english spot", "dutch rabbit", "silver fox rabbit", "french lop", "netherland dwarf rabbit", "lionhead rabbit", "florida white rabbit", "holland lop", "new zealand white rabbit", "beveren rabbit", "polish rabbit", "rex rabbit", "tan rabbit", "himalayan rabbit", "cashmere lop", "british giant rabbit", "german rabbit", "meissner lop", "alaska rabbit", "american chinchilla rabbit", "american fuzzy lop", "argente de champagne", "beige rabbit", "big silver marten rabbit", "czech red rabbit", "Fauve de Bourgogne rabbit", "flemish giant rabbit", "giant chinchilla rabbit", "gotland rabbit", "harlequin rabbit", "havana rabbit", "japanese white rabbit", "jersey wooly rabbit", "rhinelander rabbit", "siberian rabbit", "swedish hare", "english lop", "jersey wooly", "checkered giant rabbit", "mini rex", "blan de hotot", "belgian hare", "flemish giant rabbit", "mini lop", "californian rabbit", "american rabbit", "english spot", "dutch rabbit", "silver fox rabbit", "french lop", "netherland dwarf rabbit", "lionhead rabbit", "florida white rabbit", "holland lop", "new zealand white rabbit", "beveren rabbit", "polish rabbit", "rex rabbit", "tan rabbit", "himalayan rabbit", "cashmere lop", "british giant rabbit", "german rabbit", "meissner lop", "alaska rabbit", "முயல்", "华南兔", "hainan hare", "Leporidae", "cottontail rabbit"]
    
for i in tqdm_notebook(search_terms):
    driver_path = "/Users/shashankkalanithi/Documents/Data Science/Python Practice:Coding Examples/Instagram Comment Scraper/Instagram-Comments-Scraper-master/chromedriver"
    search_term = i
    print(i)
    search_and_download(
        search_term=search_term, 
        driver_path=driver_path,
        number_images=100, # You can only extract 100 images a query
    )

dalmation rabbit
Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
thrianta
Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
ERROR - Could not save https://arba.net/wp-content/uploads/2018/12/Thrianta-300x259.jpg - cannot identify image file <_io.BytesIO object at 0x7f86b00188b0>
ERROR - Could not save https://quizlet.com/cdn-cgi/image/f=auto,fit=cover,h=200,onerror=redirect,w=240/https://o.quizlet.com/cz0u.0Ua.AH8Tx7LE5OlLQ.jpg - cannot identify image file <_io.BytesIO object at 0x7f86aff978b0>
velveteen lop rabbit
Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
ERROR - Could not save https://www.elmwoodparkzoo.org/wp-content/uploads/2018/10/1920x1080-lop-rabbit.jpg - cannot identify image file <_io.BytesIO object at 0x7f86b00188b0>



In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import io
from PIL import Image
import hashlib
import time

In [48]:
###***Original Code from Anand Suresh***###

#https://gist.github.com/Anandsure/6f62d54ddf40108fdc1a6aeae8c28c75#file-img_links-py


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        #print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)